**1. Kütüphanelerin ve Veri Kümesinin Yüklenmesi**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score

# FutureWarning uyarılarını kapatıyoruz
warnings.filterwarnings('ignore', category=FutureWarning)

# Veri kümesini okuyoruz
data = pd.read_csv('/Users/efetunc/Desktop/a/c.csv', low_memory=False)



*Gerekli kütüphaneleri, modülleri ve veri kümesini projemize yükledik, ayrıca warnings.filterwarnings ile uyarılarını kapattık.*

**2. Veri Temizleme ve Ön İşleme**

In [ ]:
# Sonsuz (inf) değerleri NaN ile değiştiriyoruz
data.replace([np.inf, -np.inf], np.nan, inplace=True)

# 'price' ve 'service fee' sütunlarını sayısal formata dönüştürüyoruz
def convert_to_numeric(column):
    if column.dtype == 'object':
        column = column.str.replace('$', '', regex=False)
        column = column.str.replace(',', '', regex=False)
    return pd.to_numeric(column, errors='coerce')

data['price'] = convert_to_numeric(data['price'])
data['service fee'] = convert_to_numeric(data['service fee'])

# Eksik verileri kontrol ediyoruz
print("Eksik veriler öncesi:\n", data.isnull().sum())

# Sayısal sütunlar için eksik verileri ortalamaları ile dolduruyoruz
numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns
for col in numeric_cols:
    data[col].fillna(data[col].mean(), inplace=True)

# Kategorik sütunlar için eksik verileri modları ile dolduruyoruz
categorical_cols = data.select_dtypes(include=['object']).columns
for col in categorical_cols:
    data[col].fillna(data[col].mode()[0], inplace=True)

# Eksik verileri son kez kontrol ediyoruz
print("\nEksik veriler sonrası:\n", data.isnull().sum())


*Sonsuz değerleri NaN olacak şekilde değiştiriyoruz, price ve service fee sütunlarını sayısal formata dönüştürüyoruz. Eksik verileri hem sayısal hem de kategorik sütunlar için uygun değerlerle dolduruyoruz.*

**3. Veri Görselleştirme**

In [ ]:
# Dağılım grafiği
plt.figure(figsize=(10,6))
sns.countplot(x='neighbourhood group', data=data)
plt.title('Neighbourhood Group Dağılımı')
plt.show()

# Fiyat dağılımı grafiği
plt.figure(figsize=(10,6))
sns.histplot(data['price'], bins=30, kde=True)
plt.title('Fiyat Dağılımı')
plt.show()

# Korelasyon matrisini sayısal verilerle gösterim grafiği
numeric_data = data.select_dtypes(include=['float64', 'int64'])
plt.figure(figsize=(12,8))
sns.heatmap(numeric_data.corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Korelasyon Matrisi')
plt.show()


*Veri kümesinin dağılımını ve korelasyon matrisini görselleştiriyoruz. neighbourhood group dağılımı, fiyat dağılımı ve sayısal veriler arasındaki korelasyonlar görselleştiriliyoruz.*

**4. Kategorik Değişkenlerin Dönüştürülmesi**

In [ ]:
# Label Encoding
le = LabelEncoder()
if 'host_identity_verified' in data.columns:
    data['host_identity_verified'] = le.fit_transform(data['host_identity_verified'])
else:
    print("host_identity_verified sütunu bulunamadı.")

if 'neighbourhood group' in data.columns:
    data['neighbourhood group'] = le.fit_transform(data['neighbourhood group'])
else:
    print("neighbourhood group sütunu bulunamadı.")


*Kategorik değişkenleri sayısal formata dönüştürmek için LabelEncoder kullanıyoruz.*

**5. Özelliklerin ve Hedef Değişkenin Ayrılması**

In [ ]:
# Özellikleri ve hedef değişkeni ayırdık
X = data[['host_identity_verified', 'neighbourhood group', 'lat', 'long']]
y = data['price']

# Veriyi eğitim ve test kümelerine böldük
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Verileri ölçeklendirdik
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Eğitim ve test verilerini kontrol ettik
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


*Özellikleri (X) ve hedef değişkeni (y) ayırıyor, veriyi eğitim ve test kümelerine bölüyoruz. Verileri standartlaştırmak için StandardScaler kullanıyoruz.*

**6. Lineer Regresyon Modeli**

In [ ]:
# Lineer Regresyon Modeli
model = LinearRegression()
cv_scores_lr = cross_val_score(model, X_train, y_train, cv=5)  # Çapraz doğrulama
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
print("\nLineer Regresyon:")
print("Çapraz Doğrulama Ortalama Skoru:", np.mean(cv_scores_lr))
print("Eğitim MSE:", mean_squared_error(y_train, y_train_pred))
print("Test MSE:", mean_squared_error(y_test, y_test_pred))
print("Eğitim R^2:", r2_score(y_train, y_train_pred))
print("Test R^2:", r2_score(y_test, y_test_pred))


*Lineer regresyon modeli oluşturuyoruz, modelin çapraz doğrulama skorunu ve hem eğitim hem de test setleri için performans metriklerini hesaplıyoruz.*

**7. Ridge Regresyon Modeli ve Hiperparametre Optimizasyonu**

In [ ]:
# Ridge Regresyon Modeli ve Hiperparametre Optimizasyonu
ridge_model = Ridge()
param_grid_ridge = {'alpha': [0.1, 1, 10]}
grid_search_ridge = GridSearchCV(estimator=ridge_model, param_grid=param_grid_ridge, cv=5, n_jobs=-1, verbose=2)
grid_search_ridge.fit(X_train, y_train)
print("\nRidge Regresyon En İyi Parametreler:", grid_search_ridge.best_params_)
best_ridge = grid_search_ridge.best_estimator_
y_train_pred_best = best_ridge.predict(X_train)
y_test_pred_best = best_ridge.predict(X_test)
cv_scores_ridge = cross_val_score(best_ridge, X_train, y_train, cv=5)  # Çapraz doğrulama
print("Ridge Regresyon Çapraz Doğrulama Ortalama Skoru:", np.mean(cv_scores_ridge))
print("Ridge Regresyon Eğitim MSE:", mean_squared_error(y_train, y_train_pred_best))
print("Ridge Regresyon Test MSE:", mean_squared_error(y_test, y_test_pred_best))
print("Ridge Regresyon Eğitim R^2:", r2_score(y_train, y_train_pred_best))
print("Ridge Regresyon Test R^2:", r2_score(y_test, y_test_pred_best))


*Ridge regresyon modeli ile hiperparametre optimizasyonu yapıyoruz. En iyi hiperparametreleri bulmak için GridSearchCV kullanıyoruz ve modelin performansını değerlendiriyoruz.*

**8. Random Forest Modeli ve Hiperparametre Optimizasyonu**

In [ ]:
# Random Forest Model ve Hiperparametre Optimizasyonu
rf_model = RandomForestRegressor(random_state=42)
param_grid_rf = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20],
    'min_samples_split': [2, 5]
}
grid_search_rf = GridSearchCV(estimator=rf_model, param_grid=param_grid_rf, cv=5, n_jobs=-1, verbose=2)
grid_search_rf.fit(X_train, y_train)
print("\nRandom Forest En İyi Parametreler:", grid_search_rf.best_params_)
best_rf = grid_search_rf.best_estimator_
y_train_pred_rf = best_rf.predict(X_train)
y_test_pred_rf = best_rf.predict(X_test)
cv_scores_rf = cross_val_score(best_rf, X_train, y_train, cv=5)  # Çapraz doğrulama
print("Random Forest Çapraz Doğrulama Ortalama Skoru:", np.mean(cv_scores_rf))
print("Random Forest Eğitim MSE:", mean_squared_error(y_train, y_train_pred_rf))
print("Random Forest Test MSE:", mean_squared_error(y_test, y_test_pred_rf))
print("Random Forest Eğitim R^2:", r2_score(y_train, y_train_pred_rf))
print("Random Forest Test R^2:", r2_score(y_test, y_test_pred_rf))


*Random Forest modelini oluşturuyoruz, hiperparametre optimizasyonu yapıyor ve modelin performansını değerlendiriyoruz*

**9. Gradient Boosting Modeli ve Hiperparametre Optimizasyonu**

In [ ]:
# Gradient Boosting Modeli ve Hiperparametre Optimizasyonu
gb_model = GradientBoostingRegressor(random_state=42)
param_grid_gb = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5]
}
grid_search_gb = GridSearchCV(estimator=gb_model, param_grid=param_grid_gb, cv=5, n_jobs=-1, verbose=2)
grid_search_gb.fit(X_train, y_train)
print("\nGradient Boosting En İyi Parametreler:", grid_search_gb.best_params_)
best_gb = grid_search_gb.best_estimator_
y_train_pred_gb = best_gb.predict(X_train)
y_test_pred_gb = best_gb.predict(X_test)
cv_scores_gb = cross_val_score(best_gb, X_train, y_train, cv=5)  # Çapraz doğrulama
print("Gradient Boosting Çapraz Doğrulama Ortalama Skoru:", np.mean(cv_scores_gb))
print("Gradient Boosting Eğitim MSE:", mean_squared_error(y_train, y_train_pred_gb))
print("Gradient Boosting Test MSE:", mean_squared_error(y_test, y_test_pred_gb))
print("Gradient Boosting Eğitim R^2:", r2_score(y_train, y_train_pred_gb))
print("Gradient Boosting Test R^2:", r2_score(y_test, y_test_pred_gb))


*Gradient Boosting modeli oluşturuyoruz, hiperparametre optimizasyonu yapıyor ve modelin performansını değerlendiriyoruz.*

**10. Kümeleme Analizi: K-Means ve Agglomerative Clustering**

In [ ]:
# K-Means Kümeleme
kmeans = KMeans(n_clusters=3, random_state=42)
data['kmeans_cluster'] = kmeans.fit_predict(X)
print("\nK-Means Kümeleme Sonuçları:\n", data['kmeans_cluster'].value_counts())

# Agglomerative Clustering
agg_clustering = AgglomerativeClustering(n_clusters=3)
data['agg_cluster'] = agg_clustering.fit_predict(X)
print("\nAgglomerative Clustering Sonuçları:\n", data['agg_cluster'].value_counts())

# Silhouette Skoru hesaplama
silhouette_kmeans = silhouette_score(X, data['kmeans_cluster'])
silhouette_agg = silhouette_score(X, data['agg_cluster'])
print("\nK-Means Silhouette Skoru:", silhouette_kmeans)
print("Agglomerative Clustering Silhouette Skoru:", silhouette_agg)


*K-Means ve Agglomerative Clustering algoritmalarını uyguluyoruz, her iki yöntem için kümeleme sonuçlarını değerlendiriyoruz ve Silhouette skorlarını hesaplıyoruz.*

**11. Yorumlar**

*Bu proje sırasında, veri eksikliklerini nasıl yönetebileceğimizi ve farklı algoritmaların nasıl performans gösterdiğini öğrendik. Random Forest ve Gradient Boosting gibi yöntemlerin, daha basit modellere göre ne kadar etkili olduğunu görmek bi tecrübe oldu. Ayrıca, doğru model ve ayar seçiminin ne kadar kritik olduğunu fark ettik. Genel olarak, bu işin ne kadar dinamik, bazen zor ve keşfetmeye açık bir alan olduğunu bir kez daha deneyimledik.*